# Mushroom Classification Project (15 Epoch Optimized Version)
This notebook contains two models:
1. **Model 1**: CNN classifier with Softmax (optimized for 15 epochs)
2. **Model 2**: CNN feature extractor + Linear SVM

In [12]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [13]:
# Load dataset
data_dir = 'Mushrooms'
classes = ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius','Entoloma','Hygrocybe','Lactarius', 'Russula', 'Suillus']
img_height, img_width = 160, 160

images, labels = [], []
for label in classes:
    folder = os.path.join(data_dir, label)
    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (img_width, img_height))
            images.append(img)
            labels.append(label)

images = np.array(images, dtype=np.float32)
mean = np.mean(images, axis=(0, 1, 2))
std = np.std(images, axis=(0, 1, 2))
images = (images - mean) / std

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_cat = to_categorical(labels_encoded)


In [14]:
x_train, x_val, y_train, y_val = train_test_split(images, labels_cat, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=12,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)


In [15]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import LeakyReLU

input_layer = Input(shape=(img_height, img_width, 3))

x = Conv2D(32, (3, 3), kernel_initializer='he_normal', kernel_regularizer=l2(1e-4), padding='same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D()(x)

x = Conv2D(64, (3, 3), kernel_initializer='he_normal', kernel_regularizer=l2(1e-4), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3, 3), kernel_initializer='he_normal', kernel_regularizer=l2(1e-4), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D()(x)

x = Flatten()(x)
x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)
output_layer = Dense(len(classes), activation='softmax')(x)

model1 = Model(inputs=input_layer, outputs=output_layer)
optimizer = Adam(learning_rate=0.0005)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy()])
model1.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 160, 160, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 160, 160, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 160, 160, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 80, 80, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 80, 80, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 80, 80, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 80, 80, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 40, 40, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 40, 40, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 40, 40, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 40, 40, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,649,033 (25.36 MB)

 Trainable params: 6,648,585 (25.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [16]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

history = model1.fit(datagen.flow(x_train, y_train, batch_size=32),
                     validation_data=(x_val, y_val),
                     epochs=15,
                     callbacks=callbacks)

model1.save("model1_softmax_15epoch.h5")


C:\Users\Dhruv\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 175s 1s/step - categorical_accuracy: 0.2007 - loss: 15.5887 - val_categorical_accuracy: 0.2733 - val_loss: 7.8816 - learning_rate: 5.0000e-04
Epoch 2/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 211s 1s/step - categorical_accuracy: 0.2551 - loss: 6.1838 - val_categorical_accuracy: 0.2435 - val_loss: 6.1537 - learning_rate: 5.0000e-04
Epoch 3/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 172s 1s/step - categorical_accuracy: 0.2491 - loss: 4.7662 - val_categorical_accuracy: 0.2859 - val_loss: 4.1298 - learning_rate: 5.0000e-04
Epoch 4/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 168s 1000ms/step - categorical_accuracy: 0.2828 - loss: 3.7784 - val_categorical_accuracy: 0.3179 - val_loss: 3.2498 - learning_rate: 5.0000e-04
Epoch 5/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 155s 923ms/step - categorical_accuracy: 0.2913 - loss: 3.2503 - val_categorical_accuracy: 0.3105 - val_loss: 3.1538 - learning_rate: 5.0000e-04
Epoch 6/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - categorical_accuracy: 0.3210 - l

In [17]:
feature_extractor = Model(inputs=model1.input, outputs=model1.get_layer(index=-3).output)

x_train_feat = feature_extractor.predict(x_train)
x_val_feat = feature_extractor.predict(x_val)

y_train_svm = np.argmax(y_train, axis=1)
y_val_svm = np.argmax(y_val, axis=1)

svm_model = SVC(kernel='linear')
svm_model.fit(x_train_feat, y_train_svm)

y_pred = svm_model.predict(x_val_feat)
acc = accuracy_score(y_val_svm, y_pred)
print(f"SVM Model Accuracy: {acc:.4f}")


168/168 ━━━━━━━━━━━━━━━━━━━━ 28s 163ms/step
42/42 ━━━━━━━━━━━━━━━━━━━━ 6s 152ms/step
SVM Model Accuracy: 0.4557
